In [1]:
import requests
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
import json
from bs4 import BeautifulSoup

In [2]:
def mcd_scrape(url):
    response = requests.get(url)
    S = BeautifulSoup(response.text, 'lxml')
    mcd = []
    list_items = S.find_all('li')
    for L in list_items:
        if "McDonald's in" in L.text:
            addr = L.text.replace("  ","")
            addr = addr.replace("\n","")
            addr = addr.split("McDonald's in")
            mcd.append(addr)
    mcd2 = pd.DataFrame(mcd).drop(0,axis=1).values.tolist()

    mcd3 = []
    for elem in mcd2:
        mcd3.extend(elem)
    mcd4 = []
    for elem in mcd3:
        if elem != None:
            mcd4.append(elem)
    mcd5 = pd.DataFrame(mcd4)
    mcd6 = mcd5[0].str.split(' ',2, expand = True)
    mcd7 = mcd6[2].str.split(' ',1, expand = True).drop(0,axis=1)
    mcd8=mcd6[1].values.tolist()
    mcd9=mcd7.values.tolist()
    mcd9_list = []
    for elem in mcd9:
        mcd9_list.extend(elem)
    mcd10 = pd.DataFrame()
    mcd10['address'] = mcd9_list
    mcd10['zip'] = mcd8
    return(mcd10)

In [3]:
mo_mcd = mcd_scrape("https://www.menuism.com/restaurant-locations/mcdonalds-21019/us/mo")
mo_mcd

,address,zip
0,3996 Jeffco Blvd,63010
1,130 Arnold Crossroads Ctr,63010
2,2201 Michigan Ave,63010
3,2005 Richardson Rd,63010
4,3996 Jeffco Blvd,63010
...,...,...
583,1315 Preacher Roe Blvd,65775
584,602 S Harris St,65793
585,607 N Harris St,65793
586,602 S Harris St,65793


In [4]:
il_mcd = mcd_scrape("https://www.menuism.com/restaurant-locations/mcdonalds-21019/us/il")
il_mcd

,address,zip
0,501 W Lake St,60101-2306
1,410 W North Ave,60101-4912
2,501 W Lake St,60101-2306
3,410 W North Ave,60101-4912
4,125 S Randall Rd,60102
...,...,...
1225,12750 New York 34,60560
1226,2028 Sheridan Rd,60099
1227,2008 N Sheridan Rd,60099-4119
1228,2028 Sheridan Rd,60099


In [5]:
both = [mo_mcd,il_mcd]
both_mcd = pd.concat(both)
both_mcd
cleaned_mcd = pd.DataFrame(both_mcd['zip'].str.split('-').str[0])
cleaned_mcd['address']= both_mcd['address']
cleaned_mcd = cleaned_mcd[['address', 'zip']]
cleaned_mcd

,address,zip
0,3996 Jeffco Blvd,63010
1,130 Arnold Crossroads Ctr,63010
2,2201 Michigan Ave,63010
3,2005 Richardson Rd,63010
4,3996 Jeffco Blvd,63010
...,...,...
1225,12750 New York 34,60560
1226,2028 Sheridan Rd,60099
1227,2008 N Sheridan Rd,60099
1228,2028 Sheridan Rd,60099


In [6]:
#insert code here that drops all rows in cleaned_mcd that don't contain the relevant zip codes

zips = []
with open('Resources/zip_list.txt', 'r') as f:
    zips.extend(f.read().split('\n'))
zips

['63001',
 '63005',
 '63006',
 '63011',
 '63017',
 '63021',
 '63022',
 '63024',
 '63025',
 '63026',
 '63031',
 '63032',
 '63033',
 '63034',
 '63038',
 '63040',
 '63042',
 '63043',
 '63044',
 '63045',
 '63074',
 '63088',
 '63099',
 '63105',
 '63114',
 '63117',
 '63119',
 '63121',
 '63122',
 '63123',
 '63124',
 '63125',
 '63126',
 '63127',
 '63128',
 '63129',
 '63130',
 '63131',
 '63132',
 '63133',
 '63134',
 '63135',
 '63136',
 '63137',
 '63138',
 '63140',
 '63141',
 '63143',
 '63144',
 '63145',
 '63146',
 '63151',
 '63167',
 '63198']

In [7]:
cleaned_mcd['location'] = cleaned_mcd[['address', 'zip']].agg(' '.join, axis=1)
cleaned_mcd

,address,zip,location
0,3996 Jeffco Blvd,63010,3996 Jeffco Blvd 63010
1,130 Arnold Crossroads Ctr,63010,130 Arnold Crossroads Ctr 63010
2,2201 Michigan Ave,63010,2201 Michigan Ave 63010
3,2005 Richardson Rd,63010,2005 Richardson Rd 63010
4,3996 Jeffco Blvd,63010,3996 Jeffco Blvd 63010
...,...,...,...
1225,12750 New York 34,60560,12750 New York 34 60560
1226,2028 Sheridan Rd,60099,2028 Sheridan Rd 60099
1227,2008 N Sheridan Rd,60099,2008 N Sheridan Rd 60099
1228,2028 Sheridan Rd,60099,2028 Sheridan Rd 60099


In [ ]:
#write code that searches addresses in cleaned_mcd on https://www.mapdevelopers.com/geocode_tool.php and scrapes
    #latitude and longitude data